En este proyecto la idea es hacer un programa que sepa identificar un mensaje spam escrito en español.
Al no conseguir un dataset con mensajes en español para poder entrenar,
decidi traducir el mensaje a analizar y usar un dataset en ingles.

In [150]:
import csv
import pandas as pd
import numpy as np

In [151]:
![ ! -f spam.csv ] && wget https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv

In [152]:
dfspam = pd.read_csv("spam.csv", encoding='latin-1')[["v1","v2"]]

In [153]:
import string
punctuation = set(string.punctuation)

#Funcion tokenize, elimina los caracteres de puntuacion, 
#pone en minuscula todas las letras y divide las oraciones en palabras
#y a su vez cada oracion en vectores

def tokenize(sentence):
    tokens = []
    for token in sentence.split():
        new_token = []
        for character in token:
            if character not in punctuation:
                new_token.append(character.lower())
        if new_token:
            tokens.append("".join(new_token))
    return tokens

In [154]:
#Probando tokenize
dfspam.head()["v2"].apply(tokenize)

0    [go, until, jurong, point, crazy, available, o...
1                       [ok, lar, joking, wif, u, oni]
2    [free, entry, in, 2, a, wkly, comp, to, win, f...
3    [u, dun, say, so, early, hor, u, c, already, t...
4    [nah, i, dont, think, he, goes, to, usf, he, l...
Name: v2, dtype: object

In [155]:
from sklearn.model_selection import train_test_split

#Division de mensajes y etiquetas para entrenamiento y para prueba
train_text, test_text, train_labels, test_labels = train_test_split(dfspam["v2"], dfspam["v1"], stratify=dfspam["v1"])

In [156]:
#Creacion de matriz para acondicionar los datos 
from sklearn.feature_extraction.text import CountVectorizer
Vectorizer = CountVectorizer(tokenizer = tokenize, binary=True)

train_X = Vectorizer.fit_transform(train_text)
test_X = Vectorizer.transform(test_text)

train_X.shape

(4179, 8151)

In [157]:
#Libreria para la clasificacion de los mensajes
from sklearn.svm import LinearSVC
#Se entrena con los datos
classifier = LinearSVC()
classifier.fit(train_X, train_labels)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [158]:
import requests

#Esta funcion para traducir la encontre en un foro, no necesita instalar liberias por eso la uso. Puede fallar a veces
def Traduccion(source, target, text):
    parametros = {'sl': source, 'tl': target, 'q': text}
    cabeceras = {"Charset":"UTF-8","User-Agent":"AndroidTranslate/5.3.0.RC02.130475354-53000263 5.1 phone TRANSLATE_OPM5_TEST_1"}
    url = "https://translate.google.com/translate_a/single?client=at&dt=t&dt=ld&dt=qca&dt=rm&dt=bd&dj=1&hl=es-ES&ie=UTF-8&oe=UTF-8&inputm=2&otf=2&iid=1dd3b944-fa62-4b55-b330-74909a99969e"
    response = requests.post(url, data=parametros, headers=cabeceras)
    if response.status_code == 200:
        for x in response.json()['sentences']:
            return x['trans']
    else:
        return "Ocurrió un error"


In [159]:
#Determinamos el o los mensajes a clasificar 
mensaje1 = "Ha Ganado, por favor envie un mensaje al 8899"
mensaje2 = "Hola, como estas?"
mensaje3 = "Como estas Mariana espero verte esta noche"
mensaje4 = "Has ganado una entrada GRATIS, para ver a los Rolling Stone"
mensaje5 = "a las 4 voy a la quinta, te espero"

ejemplos = [
            mensaje1,
            mensaje2,
            mensaje3,
            mensaje4,
            mensaje5
]

examples = [
    mje1,
    mje2,
    mje3,
    mje4,
    mje5
]

for ej, ex in zip(ejemplos, examples):
  ex = Traduccion("es", "en", ej)

#Tokenizamos los mensajes para poder despues pasarlo a la funcion y asi clasificarlos
examples_X = Vectorizer.transform(examples)
predicciones = classifier.predict(examples_X)

In [160]:
for text, label in zip(ejemplos, predicciones):
    print(f"{label:5} - {text}")

spam  - Ha Ganado, por favor envie un mensaje al 8899
ham   - Hola, como estas?
ham   - Como estas Mariana espero verte esta noche
spam  - Has ganado una entrada GRATIS, para ver a los Rolling Stone
ham   - a las 4 voy a la quinta, te espero
